In [1]:
import selenium
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
import time

In [2]:
options = webdriver.ChromeOptions()
options.add_argument("--headless=new")

In [3]:
sufixes = [
    "St", "Ave", "Blvd", "Dr", "Rd", "Ln", 
    "Ct", "Cir", "Pl", "Way", "Ter", "Trl", 
    "Pkwy", "Plz", "Sq", "Loop", "Cres", "Aly", 
    "Hwy", "Path", "Expy", "Grv", "Hts", 
    "Mdws", "Pass", "Row", "Run", "Vw", "Walk", "Blf", 
    "Brk", "Crk", "Xing", "Dale", "Fld", 
    "Gld", "Hts", "Holw", "Isle", "Knl", 
    "Lndg", "Mill", "Nook", "Ovps", "Park", 
    "Pnes", "Pt", "Rdg", "Spgs", "Vis", 
    "Wd", "Mews", "Cl", "Esp", "Gdns", 
    "Grg", "Grn", "Mall", "Qy", "Vly", "Hbr",
    "Cv", "Bay", "Ests", "Trn", "Mnr" ,'(SAC', ' St'
]


roads = pd.read_excel('MapBookRoadIndex.xlsx')
roads = roads[roads['Road Name'] != 'Road Name']

roads['Road Name'] = roads['Road Name'].str.split(' ')

roads['Road Name'] = roads['Road Name'].apply(lambda x: x[:-1] if isinstance(x, list) else x)

def remove_suffix(words):
    if isinstance(words, list) and len(words) > 0:
        if words[-1] in sufixes:
            words = words[:-1]
    return words

roads['Road Name'] = roads['Road Name'].apply(remove_suffix)
roads['Road Name'] = roads['Road Name'].dropna()
roads['Road Name'] = roads['Road Name'].apply(remove_suffix)
roads['Road Name'] = roads['Road Name'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

roads = roads.dropna(subset=['Road Name'])


def remove_suffix(road_name, sufixes):
    for suffix in sufixes:
        
        if road_name.lower().endswith(suffix.lower()):
            road_name = road_name[:-len(suffix)].strip()
    return road_name

roads['Road Name'] = roads['Road Name'].apply(lambda x: remove_suffix(x, sufixes))
roads = roads.drop_duplicates(subset=['Road Name'])
roads = roads.reset_index(drop= True)
roads['Road Name'].iloc[0] = '1ST'
roads['Road Name'].iloc[2] = '3RD'

# please make sure to count that the number of streets in the data frame is equal to the number of streets
# to the number of streets in the excel file

In [4]:
#run the line below to see your web page
service = Service(executable_path='/Users/colefirkins/Desktop/selenium_project/chromedriver')

# unhash the line below to  run the method in headless mode
#driver = webdriver.Chrome(service=service, options=options) #Headless

#Hash out the the line below to run the method in URL mode
driver = webdriver.Chrome(service=service) #Not Headless
wait = WebDriverWait(driver, timeout=2)


# Build an empty list to append our data
data = []
total_parcels = []

# Loop through each street to get their parcel IDs.

for street in roads['Road Name'][0:10]:
        time.sleep(2)
        try:
            
            # Go to the bernco assessor website.
            driver.get('https://assessor.bernco.gov/public.access/Search/Disclaimer.aspx?FromUrl=../search/commonsearch.aspx?mode=realprop')
            
            # Click the aggreeeee button to get started
            login_form = driver.find_element(By.XPATH, '//*[@id="btAgree"]')
            wait.until(lambda d : login_form.is_displayed())
            login_form.click()
            
            # Find the search bar on the page
            search_bar = driver.find_element(By.XPATH, '//*[@id="inpStreet"]')
            wait.until(lambda d : search_bar.is_displayed())
            time.sleep(2)
            
            #Search for a street in our list
            search_bar.send_keys(street)
            search_bar.send_keys(Keys.RETURN)                                                 
            
            divisor = 50
            
            # Find the number of entries
            number = driver.find_element(By.XPATH, '//*[@id="frmMain"]/table/tbody/tr/td/div/div/table[2]/tbody/tr/td/table/tbody/tr[3]/td/center/table[*]/tbody/tr/td[3]/font/b[2]').text
            number = int(number)
            total_parcels.append(number)
            
            
            # Ensure the number is properly divisable by 50 to get all the pages we need!
            def adjust_number(number, divisor):
                if number % divisor != 0:
                    number = number // divisor
                    number += 1
                    number = number * 50
                else:
                    number = number
                return number

            number = adjust_number(number = number, divisor = divisor)
            

            
            # Select 50 entries per page
            fifty = driver.find_element(By.XPATH, '//*[@id="selPageSize"]/option[5]')
            fifty.click()
            
            # Click and sort the data to ascending order this also iniciates the 50 per page                                    
            per_page = driver.find_element(By.XPATH, '//*[@id="searchResults"]/thead/tr[1]/th[1]')
            per_page.click()
            
            # Loop through each page and grab the table
            for i in range((number // 50)):
                
                # Find the table of items
                parcel_table = driver.find_element(By.XPATH, '//*[@id="searchResults"]/tbody')
                # Get all the tr tags in the table
                rows = parcel_table.find_elements(By.TAG_NAME, 'tr')
                
                #row_data = row.text.split()  # You can adjust the split() to fit the structure of your table
                #data.append(row_data)
            
                #Get the text from each tag in the table
                for row in rows:
                    row_data = row.text.split()  
                    data.append(row_data)
                    # Split row text by whitespace or other delimiter
                    
                #try to go to the next button if it exists.    
                try:
                
                    # Find the "Next" button using the last <a> element in the specific section
                    next_button = driver.find_element(By.XPATH, '//*[@id="frmMain"]/table/tbody/tr/td/div/div/table[2]/tbody/tr/td/table/tbody/tr[3]/td/center/table/tbody/tr/td[2]/font[2]/a[last()]')
                    next_button.click()
                
                except:
                    pass
                    
        except:  
            pass
                    
                    
            
           
        
#Get the parcel Id from the data        
parcel_ids = [sublist[0] for sublist in data]
 
#Append the parcel Id to a dataframe
df = pd.DataFrame()
df['parcel ids'] = parcel_ids
            

driver.quit()

In [17]:
driver.quit()

In [6]:
duplicated_parcels = df[df.duplicated()]
len(duplicated_parcels)

0

In [7]:
df

,parcel ids
0,437450
1,425400
2,420227
3,101606301537620825
4,101606300638220801
...,...
159,101405722919032709
160,101405722818332708
161,101405722717932707
162,101405722021233502


In [8]:
df['parcel ids'] = df['parcel ids'].drop_duplicates()

In [9]:
df

,parcel ids
0,437450
1,425400
2,420227
3,101606301537620825
4,101606300638220801
...,...
159,101405722919032709
160,101405722818332708
161,101405722717932707
162,101405722021233502


In [10]:
total_parcels

[164]

In [11]:
sum(total_parcels)

164